# Diferenciación Automática con ``torch.autograd``

Al entrenar redes neuronales, el algoritmo más frecuentemente usado es
**back propagation**. En este algoritmo, los parámetros (pesos del modelo) se
ajustan de acuerdo al **gradiente** de la función de pérdida con respecto al
parámetro dado.

Para calcular esos gradientes, PyTorch tiene un motor de diferenciación automática
integrado llamado ``torch.autograd``. Soporta el cálculo automático del gradiente para cualquier
gráfico computacional.

Considera la red neuronal más simple de una capa, con entrada ``x``, parámetros ``w`` y ``b``, 
y alguna función de pérdida. Puede ser definida en PyTorch de la siguiente manera:

In [ ]:
import torch

x = torch.ones(5)  # tensor de entrada
y = torch.zeros(3)  # salida esperada
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [ ]:
print(z)
print(y)

In [ ]:
print(loss)

En esta red, ``w`` y ``b`` son **parámetros**, los cuales necesitamos
optimizar. Por lo tanto, necesitamos poder calcular los gradientes de la
función de pérdida con respecto a esas variables. Para hacer eso, establecemos
la propiedad ``requires_grad`` de esos tensores.

> Puedes establecer el valor de ``requires_grad`` al crear un tensor, o después usando el método ``x.requires_grad_(True)``.

Una función que aplicamos a los tensores para construir el gráfico computacional
es de hecho un objeto de la clase ``Function``. Este objeto sabe cómo calcular la función
en la dirección *hacia adelante*, y también cómo calcular su derivada durante el paso
de *propagación hacia atrás*. Una referencia a la función de propagación hacia atrás se
almacena en la propiedad ``grad_fn`` de un tensor. Puedes encontrar más información sobre ``Function`` en la [documentación](https://pytorch.org/docs/stable/autograd.html#function).

Para optimizar los pesos de los parámetros en la red neuronal, necesitamos
calcular las derivadas de nuestra función de pérdida con respecto a los parámetros,
es decir, necesitamos $\frac{\partial loss}{\partial w}$ y $\frac{\partial loss}{\partial b}$
bajo algunos valores fijos de ``x`` y ``y``. Para calcular esas derivadas, llamamos
``loss.backward()``, y luego recuperamos los valores de ``w.grad`` y ``b.grad``.

In [ ]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

> **Nota**
> - Solo podemos obtener las propiedades ``grad`` para los nodos hoja del gráfico computacional, que tienen la propiedad ``requires_grad`` establecida en ``True``. Para todos los otros nodos en nuestro gráfico, los gradientes no estarán disponibles.
> - Solo podemos realizar cálculos de gradiente usando ``backward`` una vez en un gráfico dado por razones de rendimiento. Si necesitamos hacer varias llamadas ``backward`` en el mismo gráfico, necesitamos pasar ``retain_graph=True`` a la llamada ``backward``.

In [ ]:
loss.backward()
print(w.grad)
print(b.grad)


## Deshabilitando el Seguimiento del Gradiente

Por defecto, todos los tensores con ``requires_grad=True`` están siguiendo su
historial computacional y soportan el cálculo del gradiente. Sin embargo, hay algunos casos donde no necesitamos
hacer eso, por ejemplo, cuando hemos entrenado el modelo y solo queremos aplicarlo a
algunos datos de entrada, es decir, solo queremos hacer cálculos *hacia adelante* a través de la red.
Podemos detener el rastreo de los cálculos envolviendo nuestro código de cálculo con el bloque ``torch.no_grad()``:

Otra forma de lograr el mismo resultado es usando el método ``detach()`` en el tensor:

El tensor resultante no tiene ``requires_grad=True``

Hay razones por las que podrías querer deshabilitar el seguimiento del gradiente:
  - Para marcar algunos parámetros en tu red neuronal como **parámetros congelados**.
  - Para **acelerar los cálculos** cuando solo estás haciendo un paso hacia adelante, porque los cálculos en tensores que no rastrean gradientes serían más eficientes.

In [ ]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

Otra forma de lograr el mismo resultado es usando el método ``detach()`` en el tensor:

In [ ]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

---

## Más sobre el Gráfico Computacional

Conceptualmente, autograd mantiene un registro de datos (tensores) y todas las operaciones ejecutadas
(junto con los nuevos tensores resultantes) en un gráfico acíclico dirigido (DAG) que consiste de
objetos [Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function). En este DAG, las hojas
son los tensores de entrada, las raíces son los tensores de salida. Rastreando este gráfico desde las raíces hasta las hojas, 
puedes calcular automáticamente los gradientes usando la regla de la cadena.

En un paso hacia adelante, autograd hace dos cosas simultáneamente:

- ejecuta la operación solicitada para calcular un tensor resultante
- mantiene la *función de gradiente* de la operación en el DAG.

El paso hacia atrás se inicia cuando se llama ``.backward()`` en la raíz del DAG.
``autograd`` luego:

- calcula los gradientes de cada ``.grad_fn``,
- los acumula en el atributo ``.grad`` del tensor respectivo
- usando la regla de la cadena, se propaga hasta los tensores hoja.

> **Nota**
>
> **DAGs son dinámicos en PyTorch**
> Una cosa importante a notar es que el gráfico se recrea desde cero; después de cada llamada a ``.backward()``, autograd comienza a poblar un nuevo gráfico. Esto es exactamente lo que te permite usar declaraciones de flujo de control en tu modelo; puedes cambiar la forma, tamaño y operaciones en cada iteración si es necesario.

## Gradientes de Tensores y Funciones Jacobianas

En muchos casos, tenemos una función escalar de pérdida, y necesitamos calcular el gradiente 
con respecto a algunos parámetros. Sin embargo, hay casos donde la función de salida
es un tensor arbitrario. En este caso, PyTorch te permite calcular el llamado **producto Jacobiano-vector**, en lugar de la matriz Jacobiana actual.

Para un vector función $\vec{y}=f(\vec{x})$, donde $\vec{x}=\langle x_1,\ldots,x_n\rangle$ y
$\vec{y}=\langle y_1,\ldots,y_m\rangle$, un gradiente de $\vec{y}$ con respecto a
$\vec{x}$ es dado por la **matriz Jacobiana**:

$$J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)$$

En lugar de calcular la matriz Jacobiana en sí misma, PyTorch te permite calcular el **producto Jacobiano-vector** $J^T \cdot v$ para un vector dado $v$. Esto se logra
llamando a ``backward`` con $v$ como argumento. El tamaño de $v$ debería ser el mismo que
el tamaño del tensor original, con respecto al cual queremos calcular el producto:


In [ ]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"Primer llamado\n{inp.grad}")
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSegundo llamado\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nLlamado después de poner los gradientes en cero\n{inp.grad}")

Observa que cuando llamamos ``backward`` por segunda vez con los mismos argumentos, el valor de
los gradientes es diferente. Esto sucede porque al hacer la propagación hacia atrás, PyTorch
**acumula los gradientes**, es decir, el valor de los gradientes calculados se suma al atributo ``.grad`` 
de todos los nodos hoja del gráfico computacional. Si quieres calcular los gradientes apropiados, necesitas
poner la propiedad ``.grad`` en cero antes. En el entrenamiento de la vida real esto es hecho para nosotros por el optimizador.

> **Nota**
>
> Anteriormente estábamos llamando a la función ``backward()`` sin parámetros. Esto es esencialmente equivalente a 
> llamar ``backward(torch.tensor(1.0))``, lo cual es una forma útil de calcular los gradientes en el caso de 
> una función escalar, como la pérdida durante el entrenamiento de redes neuronales.

---

## Lectura Adicional

- [API de Autograd](https://pytorch.org/docs/stable/autograd.html)